## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-09-17-11-09 +0000,nyt,Live Updates: Keir Starmer’s Allies Rally Arou...,https://www.nytimes.com/live/2026/02/09/world/...
1,2026-02-09-17-10-49 +0000,nyt,Prince William Visits Saudi Arabia for Delicat...,https://www.nytimes.com/2026/02/09/world/europ...
2,2026-02-09-17-09-27 +0000,nypost,NYC home invaders steal adorable French bulldo...,https://nypost.com/2026/02/09/us-news/nyc-home...
3,2026-02-09-17-08-15 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/09/world/...
4,2026-02-09-17-05-24 +0000,nypost,Grandma Gail Rudnick and Kimberly Murstein wan...,https://nypost.com/2026/02/09/lifestyle/grandm...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2458/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
423,super,24
424,bowl,23
156,trump,19
425,halftime,16
421,bad,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
192,2026-02-09-02-55-54 +0000,nypost,Trump slams Bad Bunny’s Super Bowl halftime sh...,https://nypost.com/2026/02/08/us-news/trump-sl...,132
90,2026-02-09-13-23-18 +0000,nyt,Trump Criticizes Bad Bunny’s Super Bowl Halfti...,https://www.nytimes.com/2026/02/09/us/trump-ba...,126
145,2026-02-09-09-36-38 +0000,bbc,Bad Bunny makes history as Trump criticises 't...,https://www.bbc.com/news/articles/c394g7nnzmzo...,116
55,2026-02-09-15-34-43 +0000,nyt,The Wedding During Bad Bunny’s Super Bowl Half...,https://www.nytimes.com/2026/02/09/us/super-bo...,114
188,2026-02-09-03-08-19 +0000,nyt,"Lady Gaga, Ricky Martin and Cardi B Join Bad B...",https://www.nytimes.com/2026/02/08/us/bad-bunn...,113


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
192,132,2026-02-09-02-55-54 +0000,nypost,Trump slams Bad Bunny’s Super Bowl halftime sh...,https://nypost.com/2026/02/08/us-news/trump-sl...
146,63,2026-02-09-09-11-41 +0000,nypost,China critic and former media tycoon Jimmy Lai...,https://nypost.com/2026/02/09/world-news/china...
211,52,2026-02-09-00-57-00 +0000,nypost,Japanese PM Sanae Takaichi’s party sweeps to m...,https://nypost.com/2026/02/08/world-news/japan...
267,50,2026-02-08-19-11-00 +0000,wsj,U.K. Prime Minister Starmer’s Top Aide Resigns...,https://www.wsj.com/world/uk/u-k-prime-ministe...
160,42,2026-02-09-06-45-50 +0000,nypost,"Juan Pablo Guanipa, ally of Venezuelan opposit...",https://nypost.com/2026/02/09/world-news/venez...
76,41,2026-02-09-14-20-48 +0000,nyt,How Ilia Malinin and Team U.S.A. Held Off Japa...,https://www.nytimes.com/2026/02/08/world/how-t...
265,36,2026-02-08-19-17-15 +0000,nypost,London’s Camden Goods Yard poised to draw New ...,https://nypost.com/2026/02/08/business/londons...
56,32,2026-02-09-15-32-32 +0000,nypost,Ghislaine Maxwell pleads the Fifth at Epstein ...,https://nypost.com/2026/02/09/us-news/ghislain...
46,31,2026-02-09-16-00-19 +0000,cbc,Air Canada halts Cuba flights as Cuban airport...,https://www.cbc.ca/news/canada/travel-to-cuba-...
84,27,2026-02-09-14-03-04 +0000,nyt,"San Francisco Teachers Begin Strike, Closing S...",https://www.nytimes.com/2026/02/09/us/san-fran...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
